In [3]:
import pandas as pd
import numpy as np

# Load dataset
file_path = 'Tabel_Harga.csv'  # Ganti dengan path Anda jika berbeda
data = pd.read_csv(file_path, delimiter=';')

# Memilih kolom yang relevan
data = data[['Beras', '14,05', '14,05.1', '13,95', '14,1', '14,1.1']]
data.columns = ['Beras', 'Harga1', 'Harga2', 'Harga3', 'Harga4', 'Harga5']

# Memfilter hanya baris untuk 'Beras Kualitas Bawah I', 'Beras Kualitas Medium I', dan 'Beras Kualitas Super I'
filtered_data = data[data['Beras'].isin(['Beras Kualitas Bawah I', 'Beras Kualitas Medium I', 'Beras Kualitas Super I'])]

# Mengubah format desimal dari koma ke titik dan mengonversi ke numerik
for col in ['Harga1', 'Harga2', 'Harga3', 'Harga4', 'Harga5']:
    filtered_data[col] = filtered_data[col].str.replace(',', '.').astype(float)

print(filtered_data.head())

                     Beras  Harga1  Harga2  Harga3  Harga4  Harga5
0   Beras Kualitas Bawah I   12.85   12.85   12.75   12.90   12.90
2  Beras Kualitas Medium I   14.50   14.50   14.45   14.55   14.55
4   Beras Kualitas Super I   16.80   16.80   16.55   16.80   16.80


<ipython-input-3-9c62dfad55af>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data[col] = filtered_data[col].str.replace(',', '.').astype(float)
<ipython-input-3-9c62dfad55af>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data[col] = filtered_data[col].str.replace(',', '.').astype(float)
<ipython-input-3-9c62dfad55af>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [4]:
# Menyiapkan data deret waktu untuk forecasting
time_series_data = filtered_data.set_index('Beras').T  # Transpose agar harga berurutan sebagai series
time_series_data.reset_index(drop=True, inplace=True)

# Mengambil kolom target untuk forecasting (misalnya 'Beras Kualitas Bawah I')
target_series = time_series_data['Beras Kualitas Bawah I'].values.reshape(-1, 1)


In [6]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Normalisasi data
scaler = MinMaxScaler(feature_range=(0, 1))
target_series_scaled = scaler.fit_transform(target_series)

# Fungsi untuk membuat dataset yang sesuai untuk LSTM/GRU
def create_dataset(data, time_step=3):
    X, y = [], []
    # Adjusted loop to accommodate smaller datasets
    for i in range(len(data)-time_step-1):
        X.append(data[i:(i+time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

# Reduced time_step to 1 to increase sample size
time_step = 1
X, y = create_dataset(target_series_scaled, time_step)
X = X.reshape((X.shape[0], X.shape[1], 1))

# Split data untuk train dan test
# Reduced test_size to accommodate smaller dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [8]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Normalisasi data
scaler = MinMaxScaler(feature_range=(0, 1))
target_series_scaled = scaler.fit_transform(target_series)

# Fungsi untuk membuat dataset yang sesuai untuk LSTM/GRU
def create_dataset(data, time_step=3):
    X, y = [], []
    for i in range(len(data)-time_step-1):
        X.append(data[i:(i+time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

# Adjust time_step to 1 to allow for more samples
time_step = 1
X, y = create_dataset(target_series_scaled, time_step)
X = X.reshape((X.shape[0], X.shape[1], 1))

# Split data untuk train dan test
# Adjust test_size to be smaller to allow for a training set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # test_size reduced to ensure enough data for training

In [14]:
!pip install tensorflow

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense
from sklearn.neural_network import MLPRegressor

# Fungsi untuk evaluasi
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import math

def evaluate_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = math.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    return mae, rmse, r2

# Model LSTM
lstm_model = Sequential()
lstm_model.add(LSTM(50, input_shape=(X_train.shape[1], 1)))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mse')
lstm_model.fit(X_train, y_train, epochs=10, batch_size=1, verbose=0)
y_pred_lstm = lstm_model.predict(X_test)
mae_lstm, rmse_lstm, r2_lstm = evaluate_metrics(y_test, y_pred_lstm)

# Model GRU
gru_model = Sequential()
gru_model.add(GRU(50, input_shape=(X_train.shape[1], 1)))
gru_model.add(Dense(1))
gru_model.compile(optimizer='adam', loss='mse')
gru_model.fit(X_train, y_train, epochs=10, batch_size=1, verbose=0)
y_pred_gru = gru_model.predict(X_test)
mae_gru, rmse_gru, r2_gru = evaluate_metrics(y_test, y_pred_gru)

# Model ELM (Menggunakan MLP sebagai pendekatan ELM)
X_train_elm = X_train.reshape(X_train.shape[0], X_train.shape[1])  # Reshape ke 2D
X_test_elm = X_test.reshape(X_test.shape[0], X_test.shape[1])
elm_model = MLPRegressor(hidden_layer_sizes=(50,), activation='relu', max_iter=500)
elm_model.fit(X_train_elm, y_train)
y_pred_elm = elm_model.predict(X_test_elm)
mae_elm, rmse_elm, r2_elm = evaluate_metrics(y_test, y_pred_elm)

# Menampilkan hasil
print("\n\nLSTM - MAE: {:.4f}, RMSE: {:.4f}, R2: {:.4f}".format(mae_lstm, rmse_lstm, r2_lstm))
print("GRU  - MAE: {:.4f}, RMSE: {:.4f}, R2: {:.4f}".format(mae_gru, rmse_gru, r2_gru))
print("ELM  - MAE: {:.4f}, RMSE: {:.4f}, R2: {:.4f} \n\n".format(mae_elm, rmse_elm, r2_elm))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step


LSTM - MAE: 0.6458, RMSE: 0.6458, R2: nan
GRU  - MAE: 0.5659, RMSE: 0.5659, R2: nan
ELM  - MAE: 0.6253, RMSE: 0.6253, R2: nan 




/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [19]:
import numpy as np

# Pastikan tidak ada nilai NaN atau inf pada X_train, X_test, y_train, dan y_test
def clean_data(*arrays):
    return [np.nan_to_num(array, nan=0.0, posinf=0.0, neginf=0.0) for array in arrays]

X_train, X_test, y_train, y_test = clean_data(X_train, X_test, y_train, y_test)


In [20]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
y_train_scaled = scaler.fit_transform(y_train.reshape(-1, 1))
y_test_scaled = scaler.transform(y_test.reshape(-1, 1))

# Pastikan X_train dan X_test tetap memiliki dimensi yang benar
X_train_scaled, X_test_scaled = clean_data(X_train, X_test)


In [23]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def clean_and_scale_data(scaler, *arrays):
    """Cleans and scales the input arrays using the provided scaler.

    Args:
        scaler: The scaler object to use for scaling.
        *arrays: The arrays to clean and scale.

    Returns:
        A list of cleaned and scaled arrays.
    """
    cleaned_arrays = [np.nan_to_num(array, nan=0.0, posinf=0.0, neginf=0.0) for array in arrays]

    # Reshape before scaling for compatibility with MinMaxScaler
    reshaped_arrays = [arr.reshape(-1, 1) if arr.ndim > 2 else arr for arr in cleaned_arrays]
    # Reshape to 2D if more than 2 dimensions, otherwise keep original shape

    # Scale the data - only scale 1D or 2D arrays
    scaled_arrays = [scaler.fit_transform(arr) if (arr.ndim <= 2 and i < len(arrays) - 2) else arr
                     for i, arr in enumerate(reshaped_arrays)]
    # Assuming the last two elements are y values, not to be scaled by this function.
    # Modify this if you need to scale all inputs.

    # Reshape back to original shape if necessary
    original_shaped_arrays = [arr.reshape(arrays[i].shape) for i, arr in enumerate(scaled_arrays)]
    return original_shaped_arrays


# ... (rest of the code) ...


# Usage:
scaler = MinMaxScaler(feature_range=(0, 1))
X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled = clean_and_scale_data(scaler, X_train, X_test, y_train, y_test)

# Reshape for LSTM and GRU
X_train_scaled = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_test_scaled = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

# ... (rest of the code) ...

In [25]:
# LSTM Model
lstm_model.fit(X_train_scaled, y_train_scaled, epochs=10, batch_size=1, verbose=0)
y_pred_lstm = lstm_model.predict(X_test_scaled)
y_pred_lstm = scaler.inverse_transform(y_pred_lstm)  # Invers normalisasi

# GRU Model
gru_model.fit(X_train_scaled, y_train_scaled, epochs=10, batch_size=1, verbose=0)
y_pred_gru = gru_model.predict(X_test_scaled)
y_pred_gru = scaler.inverse_transform(y_pred_gru)

# ELM Model
elm_model.fit(X_train_scaled.reshape(X_train.shape[0], X_train.shape[1]), y_train_scaled.ravel())
y_pred_elm = elm_model.predict(X_test_scaled.reshape(X_test.shape[0], X_test.shape[1]))
y_pred_elm = scaler.inverse_transform(y_pred_elm.reshape(-1, 1))

# Evaluasi ulang hasil
mae_lstm, rmse_lstm, r2_lstm = evaluate_metrics(y_test, y_pred_lstm)
mae_gru, rmse_gru, r2_gru = evaluate_metrics(y_test, y_pred_gru)
mae_elm, rmse_elm, r2_elm = evaluate_metrics(y_test, y_pred_elm)

print("\n\nLSTM - MAE: {:.4f}, RMSE: {:.4f}, R2: {:.4f}".format(mae_lstm, rmse_lstm, r2_lstm))
print("GRU  - MAE: {:.4f}, RMSE: {:.4f}, R2: {:.4f}".format(mae_gru, rmse_gru, r2_gru))
print("ELM  - MAE: {:.4f}, RMSE: {:.4f}, R2: {:.4f} \n\n".format(mae_elm, rmse_elm, r2_elm))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


LSTM - MAE: 0.1712, RMSE: 0.1712, R2: nan
GRU  - MAE: 0.3246, RMSE: 0.3246, R2: nan
ELM  - MAE: 0.9459, RMSE: 0.9459, R2: nan 




/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
